In [41]:
import fxcmpy
import pandas as pd
import datetime as dt
import json
import ta
import warnings

In [42]:
con = fxcmpy.fxcmpy(access_token="d808da394895605e414394e79c1a3c9ffafad862", log_level='error', server='demo', log_file='log.txt')

# return connection
if con.is_connected():
    print("Connect Successful!")

Connect Successful!


In [3]:
get_data = con.get_candles("EUR/USD", period='m5', number=50)[['bidopen','bidclose','bidhigh','bidlow']]
get_data.rename(columns = {'bidopen':'Open', 'bidhigh':'High', 'bidlow': 'Low', 'bidclose': 'Close'}, inplace = True)

In [30]:
res_ta = ta.trend.MACD(get_data['Close'], window_fast=12, window_slow=26 ,window_sign=9) # signal line is base and macd line will crossover
macd_line, signal_line = res_ta.macd()[-1:].values[0], res_ta.macd_signal()[-1:].values[0]

In [31]:
if macd_line > signal_line:
    signal, arrow_type = "BUY", "arrow-up"
elif macd_line < signal_line:
    signal, arrow_type = "SELL", "arrow-down"
else:
    signal, arrow_type = "NEUTRAL", "non-arrow"



('BUY', 'arrow-up')

In [35]:
res_ta = ta.momentum.RSIIndicator(get_data['Close'][-14:], window=14)

In [36]:
res_ta.rsi()

date
2022-05-03 10:40:00          NaN
2022-05-03 10:45:00          NaN
2022-05-03 10:50:00          NaN
2022-05-03 10:55:00          NaN
2022-05-03 11:00:00          NaN
2022-05-03 11:05:00          NaN
2022-05-03 11:10:00          NaN
2022-05-03 11:15:00          NaN
2022-05-03 11:20:00          NaN
2022-05-03 11:25:00          NaN
2022-05-03 11:30:00          NaN
2022-05-03 11:35:00          NaN
2022-05-03 11:40:00          NaN
2022-05-03 11:45:00    65.117649
Name: rsi, dtype: float64

In [14]:
indicators_signal = {
    "1": [
        {
            "timeframe": "10Min",
            "value": 1.05446,
            "signal": "SELL",
            "datetime": 1651592563000,
            "type": "trend"
        },
        {
            "timeframe": "1H",
            "value": 1.05213,
            "signal": "BUY",
            "datetime": 1651592563000,
            "type": "trend"
        }
    ],
    "2": [
        {
            "timeframe": "10Min",
            "arrowtype": "arrow-down",
            "value": 0.00009,
            "signal": "SELL",
            "datetime": 1651592563000,
            "type": "trend"
        },
        {
            "timeframe": "1H",
            "arrowtype": "arrow-up",
            "value": 0.00058,
            "signal": "BUY",
            "datetime": 1651592563000,
            "type": "trend"
        }
    ],
    "3": [
        {
            "timeframe": "10Min",
            "value": 32.5403,
            "signal": "NEUTRAL",
            "datetime": 1651592563000,
            "type": "momentum"
        },
        {
            "timeframe": "1H",
            "value": 54.35956,
            "signal": "NEUTRAL",
            "datetime": 1651592564000,
            "type": "momentum"
        }
    ]
}

In [49]:
return_summary = {}
return_summary['timeframe'] = {}
return_summary['all'] = {}
return_summary['summary'] = {}
return_summary['all']['SELL'], return_summary['all']['BUY'] = 0, 0

return_summary['trend'] = {}
return_summary['trend']['BUY'] = 0
return_summary['trend']['SELL'] = 0
return_summary['trend']['NEUTRAL'] = 0

return_summary['momentum'] = {}
return_summary['momentum']['OVERBOUGHT'] = 0
return_summary['momentum']['OVERSOLD'] = 0
return_summary['momentum']['NEUTRAL'] = 0
for indicator in indicators_signal.keys():

    for tf in indicators_signal[indicator]:

        if tf['timeframe'] not in return_summary['timeframe']:
            return_summary['timeframe'][tf['timeframe']] = {}
            # initial trend 
            return_summary['timeframe'][tf['timeframe']]['trend'] = {}
            return_summary['timeframe'][tf['timeframe']]['trend']['BUY'] = 0
            return_summary['timeframe'][tf['timeframe']]['trend']['SELL'] = 0
            return_summary['timeframe'][tf['timeframe']]['trend']['NEUTRAL'] = 0
            # initial momentum
            return_summary['timeframe'][tf['timeframe']]['momentum'] = {}
            return_summary['timeframe'][tf['timeframe']]['momentum']['OVERBOUGHT'] = 0
            return_summary['timeframe'][tf['timeframe']]['momentum']['OVERSOLD'] = 0
            return_summary['timeframe'][tf['timeframe']]['momentum']['NEUTRAL'] = 0

        if tf['type'] == 'trend':
            return_summary['timeframe'][tf['timeframe']]['trend'][tf['signal']] += 1
            return_summary['trend'][tf['signal']] += 1
        elif tf['type'] == 'momentum':
            return_summary['timeframe'][tf['timeframe']]['momentum'][tf['signal']] += 1
            return_summary['momentum'][tf['signal']] += 1

        if tf['signal'] in ['BUY', 'OVERSOLD']: return_summary['all']['BUY'] += 1
        elif tf['signal'] in ['SELL', 'OVERBOUGHT']: return_summary['all']['SELL'] += 1
    
if (return_summary['all']['BUY'] + return_summary['all']['SELL']) != 0:
    return_summary['summary']['BUY'] = return_summary['all']['BUY'] * 100 / (return_summary['all']['BUY'] + return_summary['all']['SELL'])
    return_summary['summary']['SELL'] = return_summary['all']['SELL'] * 100 / (return_summary['all']['BUY'] + return_summary['all']['SELL'])
else :
    return_summary['summary']['BUY'] = 0
    return_summary['summary']['SELL'] = 0

In [50]:
return_summary

{'timeframe': {'10Min': {'trend': {'BUY': 0, 'SELL': 2, 'NEUTRAL': 0},
   'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 1}},
  '1H': {'trend': {'BUY': 2, 'SELL': 0, 'NEUTRAL': 0},
   'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 1}}},
 'all': {'SELL': 2, 'BUY': 2},
 'summary': {'BUY': 50.0, 'SELL': 50.0},
 'trend': {'BUY': 2, 'SELL': 2, 'NEUTRAL': 0},
 'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 2}}

In [29]:
high, low, close = get_data['High'], get_data['Low'], get_data['Close']


In [23]:

res_ta = ta.trend.PSARIndicator(high, low, close, step=float(0.02), max_step=float(0.2))
ta_value = res_ta.psar()[-1:].values[0]
ta_value

1.0540171746493372

In [19]:
warnings.filterwarnings("ignore")

In [37]:
res_ta = ta.trend.CCIIndicator(high, low, close, window=int(20), constant=float(0.015))
ta_value = res_ta.cci()[-1:].values[0]

In [39]:
# res_ta.cci()

In [40]:
res_ta = ta.momentum.StochasticOscillator(high, low, close, window=int(14), smooth_window=int(3))

In [44]:
indicator_value, indicators_signal = res_ta.stoch()[-1:], res_ta.stoch_signal()[-1:]
indicator_value, indicators_signal

(date
 2022-05-04 15:20:00    91.950464
 Name: stoch_k, dtype: float64,
 date
 2022-05-04 15:20:00    93.014905
 Name: stoch_k_signal, dtype: float64)

In [48]:
res_ta = ta.momentum.WilliamsRIndicator(high, low, close, lbp=14)
ta_value = res_ta.williams_r()[-1:].values[0]
ta_value

-8.049535603719619

In [3]:
print("W%R calculation error on > ")
import math

W%R calculation error on > 


In [14]:
start = dt.datetime(2022, 5, 1)
end = dt.datetime(2022, 5, 2)
data = con.get_candles("EUR/USD", period='m5', start=start, end=end)[['bidopen','bidclose','bidhigh','bidlow']]

In [17]:
data.index.view(int) ## - 162000, not more than 172800

array([1651439100000000000, 1651439400000000000, 1651439700000000000,
       1651440000000000000, 1651440300000000000, 1651440600000000000,
       1651440900000000000, 1651441200000000000, 1651442100000000000,
       1651442400000000000, 1651442700000000000, 1651443000000000000,
       1651443300000000000, 1651443600000000000, 1651443900000000000,
       1651444200000000000, 1651444500000000000, 1651444800000000000,
       1651445100000000000, 1651445400000000000, 1651446000000000000,
       1651446300000000000, 1651446600000000000, 1651446900000000000,
       1651447200000000000, 1651447500000000000, 1651447800000000000,
       1651448100000000000, 1651448400000000000, 1651448700000000000,
       1651449000000000000, 1651449300000000000, 1651449600000000000])

In [23]:
import datetime as dt

is_open = (math.floor(dt.datetime.utcnow().timestamp()) - 162000) % 604800 >= 172800
is_open

False

In [9]:
import time
import sys
import requests as r
from bs4 import BeautifulSoup as bs
import html5lib
import re
import subprocess as s


class Curr:

	def multiple_replacer(*key_values):
        replace_dict = dict(key_values)
        replacement_function = lambda match: replace_dict[match.group(0)]
        pattern = re.compile("|".join([re.escape(k) for k, v in key_values]), re.M)
        return lambda string: pattern.sub(replacement_function, string)
        
    def multiple_replace(string, *key_values):
	    return Curr.multiple_replacer(*key_values)(string)

	def strength():

		#get the site and parse the html
		x = r.get("http://www.livecharts.co.uk/currency-strength.php").text

		cur = bs(x, "html5lib")

		#Find all Currency by id
		pairs = [c for c in cur.find_all(id="map-innercontainer-symbol")]

		#To be excluded
		y = str('style="background-image:none"')

		#EURO CURRENCY index=0 or pair[0]
		#Finding the current level we need to loop through all levels

		levels = [
			'map-innercontainer-strong3',
			'map-innercontainer-strong2',
			'map-innercontainer-strong1',
			'map-innercontainer-weak1',
			'map-innercontainer-weak2',
			'map-innercontainer-weak3',
		]

		lv = [l for l in cur.find_all(id=levels)]

		mine = lv

		# #Show out the results to the user
		replacements = (u"div", u""),(u'id="map-innercontainer-',u""),(u"<",""),(u'style="background-image:none"',u""),(u'">',""),(u"/",u""),(u"",u""),(u">",u""),(u'"',u"")

		# print(f'Currency\t\tStrength\n')
		# print(f'{pairs[0].text}\t\t\t{Curr.multiple_replace(str(mine[0:6]), *replacements)}\n')
		# print(f'{pairs[1].text}\t\t\t\t{Curr.multiple_replace(str(mine[6:12]), *replacements)}\n')
		# print(f'{pairs[2].text}\t\t\t\t{Curr.multiple_replace(str(mine[12:18]), *replacements)}\n')
		# print(f'{pairs[3].text}\t\t\t\t{Curr.multiple_replace(str(mine[18:24]), *replacements)}\n')
		# print(f'{pairs[4].text}\t\t\t\t{Curr.multiple_replace(str(mine[24:30]), *replacements)}\n')
		# print(f'{pairs[5].text}\t\t\t\t{Curr.multiple_replace(str(mine[30:36]), *replacements)}\n')
		# print(f'{pairs[6].text}\t\t\t\t{Curr.multiple_replace(str(mine[36:42]), *replacements)}\n')
		# print(f'{pairs[7].text}\t\t\t\t{Curr.multiple_replace(str(mine[42:48]), *replacements)}\n')
		# print(f'{pairs[8].text}\t\t\t\t{Curr.multiple_replace(str(mine[48:54]), *replacements)}\n')
		# print(f'{pairs[9].text}\t\t\t\t{Curr.multiple_replace(str(mine[54:60]), *replacements)}\n')


pairs, mine = Curr.strength()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

In [40]:
import time
import sys
import requests as r
from bs4 import BeautifulSoup as bs
import html5lib
import re
import subprocess as s

def multiple_replacer(*key_values):
    replace_dict = dict(key_values)
    replacement_function = lambda match: replace_dict[match.group(0)]
    pattern = re.compile("|".join([re.escape(k) for k, v in key_values]), re.M)
    return lambda string: pattern.sub(replacement_function, string)
        
def multiple_replace(string, *key_values):
	return multiple_replacer(*key_values)(string)

def strength():

	#get the site and parse the html
	x = r.get("http://www.livecharts.co.uk/currency-strength.php").text

	cur = bs(x, "html5lib")

	#Find all Currency by id
	pairs = [c for c in cur.find_all(id="map-innercontainer-symbol")]

	#To be excluded
	y = str('style="background-image:none"')

	#EURO CURRENCY index=0 or pair[0]
	#Finding the current level we need to loop through all levels

	levels = [
		'map-innercontainer-strong3',
		'map-innercontainer-strong2',
		'map-innercontainer-strong1',
		'map-innercontainer-weak1',
		'map-innercontainer-weak2',
		'map-innercontainer-weak3',
	]

	lv = [l for l in cur.find_all(id=levels)]

	mine = lv

	replacements = (u"div", u""),(u'id="map-innercontainer-',u""),(u"<",""),(u'style="background-image:none"',u""),(u'">',""),(u"/",u""),(u"",u""),(u">",u""),(u'"',u"")

	strength_dict = dict()
	for i in range(10):
		pair_strength = multiple_replace(str(mine[(i)*6:(i+1)*6]), *replacements)
		pair_strength = [st.strip() for st in pair_strength.replace(u'\xa0', u' ').replace('[', '').replace(']', '').split(",")]
		# get strength number 
		num_strength = 7 - len([lowest for lowest in pair_strength if lowest == "weak3"])

		strength_dict[pairs[i].text[:3]] = num_strength

	return strength_dict

strength()

{'EUR': 3,
 'GBP': 4,
 'USD': 2,
 'AUD': 6,
 'JPY': 1,
 'CHF': 4,
 'NZD': 4,
 'CAD': 4,
 'NOK': 3,
 'DKK': 2}

In [153]:
start = dt.datetime(2022, 1, 1)
end = dt.datetime(2022, 2, 1)

get_data = con.get_candles("EUR/USD", period='m15', start=start, end=end)[['bidopen','bidclose','bidhigh','bidlow']]
get_data.rename(columns = {'bidopen':'Open', 'bidhigh':'High', 'bidlow': 'Low', 'bidclose': 'Close'}, inplace = True)

In [69]:
d = get_data[-25:]['Close']

In [62]:
d

date
2022-01-31 22:05:00    1.12322
2022-01-31 22:10:00    1.12314
2022-01-31 22:15:00    1.12311
2022-01-31 22:20:00    1.12310
2022-01-31 22:25:00    1.12308
2022-01-31 22:30:00    1.12307
2022-01-31 22:35:00    1.12309
2022-01-31 22:40:00    1.12307
2022-01-31 22:45:00    1.12307
2022-01-31 22:50:00    1.12310
2022-01-31 22:55:00    1.12303
2022-01-31 23:00:00    1.12309
2022-01-31 23:05:00    1.12318
2022-01-31 23:10:00    1.12309
2022-01-31 23:15:00    1.12299
2022-01-31 23:20:00    1.12308
2022-01-31 23:25:00    1.12302
2022-01-31 23:30:00    1.12302
2022-01-31 23:35:00    1.12288
2022-01-31 23:40:00    1.12305
2022-01-31 23:45:00    1.12306
2022-01-31 23:50:00    1.12297
2022-01-31 23:55:00    1.12284
2022-02-01 00:00:00    1.12292
Name: Close, dtype: float64

In [148]:
x = abs((d - d.shift(1))[-24:] * 10 ** 4)

In [149]:
def format(time):
    return dt.datetime.fromtimestamp(time.timestamp()).strftime("%H:%M")
x.index = x.index.map(format)

In [150]:
X = x.index.values.tolist()
Y = list(map(lambda v: round(v, 1), x.tolist()))

In [152]:
x.index.tolist()



['01:15',
 '01:30',
 '01:45',
 '02:00',
 '02:15',
 '02:30',
 '02:45',
 '03:00',
 '03:15',
 '03:30',
 '03:45',
 '04:00',
 '04:15',
 '04:30',
 '04:45',
 '05:00',
 '05:15',
 '05:30',
 '05:45',
 '06:00',
 '06:15',
 '06:30',
 '06:45',
 '07:00']

packet queue is empty, aborting


In [145]:
d.index[0].timestamp()

1643652000.0

In [164]:
import matplotlib.pyplot as plt

In [174]:
data_min, data_max = min(get_data['Close'].value_counts().index), max(get_data['Close'].value_counts().index)

In [184]:
import math

In [209]:
INTERVAL = 200

clean_data = ((get_data['Close'] * 10 ** 5).apply(math.floor) - (get_data['Close'] * 10 ** 5).apply(math.floor) % INTERVAL + math.floor(INTERVAL / 2)).value_counts().sort_index()
clean_data.index.values

array([111300, 111500, 111700, 111900, 112100, 112300, 112500, 112700,
       112900, 113100, 113300, 113500, 113700, 113900, 114100, 114300,
       114500, 114700, 114900])

In [207]:
X = clean_data.tolist()
Y = list(map(lambda p: p / 10 ** 5, clean_data.index.tolist()))

X, Y

([40,
  127,
  41,
  20,
  24,
  53,
  11,
  41,
  267,
  300,
  362,
  274,
  107,
  39,
  142,
  41,
  91,
  76,
  1],
 [1.113,
  1.115,
  1.117,
  1.119,
  1.121,
  1.123,
  1.125,
  1.127,
  1.129,
  1.131,
  1.133,
  1.135,
  1.137,
  1.139,
  1.141,
  1.143,
  1.145,
  1.147,
  1.149])